# Sistema de recomendación.

Primero, se importan las librerías necesarias.

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

## Sistema de recomendación de mejor desarrollador

In [24]:
df_sentimientos = pd.read_parquet('sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2', 'recommend', 'Año'])
df_juegos = pd.read_parquet('etljuegoslimpio.parquet', columns= ['id', 'developer'])
df = df_sentimientos.merge(df_juegos, how='left', left_on='item_id', right_on='id')
df.drop(columns=['item_id', 'id'], inplace=True)
df.dropna(inplace=True)
df = df.groupby(['Año', 'developer']).agg({'sentiment_analysis_2': (lambda x: (x == 2).sum()), 'recommend': 'sum'})
df['rating'] = df['sentiment_analysis_2'] + df['recommend']
df.drop(columns=['sentiment_analysis_2', 'recommend'], inplace=True)
df.reset_index(inplace=True)
df.head()

,Año,developer,rating
0,2010.0,1C Company,2
1,2010.0,1C: Maddox Games,1
2,2010.0,Amanita Design,1
3,2010.0,BioWare,2
4,2010.0,"BioWare,Aspyr (Mac)",2


In [3]:
df.to_parquet('mejordesarrollador.parquet')

In [7]:
def best_developer_year(año: int):

    try:
        año = int(año)
    except Exception as e:
        return {f'Error {e}': 'Debe insertar un número entero.'}
    
    df = pd.read_parquet('mejordesarrollador.parquet')
    años = list(df['Año'].unique())
    años = [int(x) for x in años]
    
    df = df[df['Año'].isin([año])]
    if len(df) == 0:
        del df
        return {'Mensaje': f'No hay registros del año {año}',
                'Los años disponibles son:': años}

    df = df.sort_values('rating', ascending=False).reset_index()
    
    result = {
                'Puesto 1': df.loc[0, 'developer'],
                'Puesto 2': df.loc[1, 'developer'],
                'Puesto 3': df.loc[2, 'developer']
            }
    del df
    return result

In [9]:
best_developer_year(4500)

{'Mensaje': 'No hay registros del año 4500',
 'Los años disponibles son:': [2010, 2011, 2012, 2013, 2014, 2015]}

## Sistema de análisis de desarrolladores por reseñas

In [25]:
df_juegos['developer'] = df_juegos['developer'].apply(lambda x: x.lower())
df_sentimientos2 = pd.read_parquet('sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2'])
df2 = df_sentimientos2.merge(df_juegos,how= 'left', left_on= 'item_id', right_on='id')
del df_sentimientos2
del df_juegos
df2.drop(columns=['id', 'item_id'], inplace=True)
df2.head()

,sentiment_analysis_2,developer
0,2,tripwire interactive
1,2,ace team
2,2,NaN
3,2,NaN
4,2,scs software


In [26]:
# Crear las columnas 'positivos' y 'negativos'
df2['positivos'] = df2['sentiment_analysis_2'].apply(lambda x: 1 if x == 2 else 0)
df2['negativos'] = df2['sentiment_analysis_2'].apply(lambda x: 1 if x == 0 else 0)

# Eliminar la columna 'sentiment_analysis_2'
df2.drop(columns='sentiment_analysis_2', inplace=True)

# Agrupar por la columna 'developer' y sumar
df2 = df2.groupby('developer').agg('sum').reset_index()

# Mostrar las primeras filas del DataFrame
print(df2.head())


                               developer  positivos  negativos
0                         07th expansion          0          1
1  10th art studio,adventure productions          0          0
2                             10tons ltd          1          0
3                         11 bit studios         27         29
4                               14° east          1          1


In [27]:
df2.to_parquet('analisisreseñasdesarrolladores.parquet')

In [28]:
def developer_reviews_analysis(desarrolladora: str):

    if not isinstance(desarrolladora, str):
        return {'Mensaje': 'Debe ingresar una cadena de texto'}
    
    desarrolladora = desarrolladora.lower()
    
    df3 = pd.read_parquet('analisisreseñasdesarrolladores.parquet')
    
    developers = list(df['developer'].unique()) 

    df3 = df3[df3['developer'].isin([desarrolladora])].reset_index()
    if len(df) == 0:
        del df3
        return {f'Desarrolladora no encontrada: {desarrolladora}.': f'Desarrolladoras disponibles {", ".join(developers)}'}

    resultado = {
        desarrolladora: [f'Negative = {int(df3["negativos"].values[0])}', f'Positive = {int(df3["positivos"].values[0])}']
    }

    del df3

    return resultado

In [31]:
def developer_reviews_analysis(desarrolladora):
    # Assuming df2 is the DataFrame with 'positivos' and 'negativos' columns
    df2 = pd.read_parquet('analisisreseñasdesarrolladores.parquet')

    # Filter the DataFrame based on the developer
    df3 = df2[df2['developer'] == desarrolladora]

    # If the DataFrame is empty, return a message with available developers
    if df3.empty:
        developers = df2['developer'].unique()
        return {
            f'Desarrolladora no encontrada: {desarrolladora}.':
            f'Desarrolladoras disponibles: {", ".join(developers)}'
        }

    # Prepare the result dictionary
    resultado = {
        desarrolladora: [
            f'Negative = {int(df3["negativos"].values[0])}',
            f'Positive = {int(df3["positivos"].values[0])}'
        ]
    }

    return resultado

# Example usage
print(developer_reviews_analysis('asereje'))


{'Desarrolladora no encontrada: asereje.': 'Desarrolladoras disponibles: 07th expansion, 10th art studio,adventure productions, 10tons ltd, 11 bit studios, 14° east, 16bit nights, 17-bit, 1c company, 1c-avalon, 1c-softclub, 1c: maddox games, 1c:inoco, 22cans, 24 caret games, 2d boy, 2darray, 2dengine.com, 2k australia,gearbox software,aspyr (mac &amp; linux), 2k australia,gearbox software,aspyr (mac and linux), 2k australia,gearbox software,aspyr (mac,linux), 2k boston,2k australia, 2k boston,2k australia,blind squirrel,feral interactive (mac), 2k czech,feral interactive (mac), 2k marin, 2k marin,2k china,digital extremes,2k australia,feral interactive (mac), 2play, 2x2 games, 3 sprockets, 30.06 studios ltd, 3000ad, 343 industries,vanguard games, 3909, 3d realms (apogee software), 3f interactive, 4 i lab, 48h studio, 49 games, 4a games, 5 bits games, 5 lives studios, 5th cell media, 5wolf, 773, 777 studios, 8-bit studio, 800 north and digital ranch, 8888888 labs, 88mm, 8monkey labs, 9h

### Se desarrolla el archivo y la función para desarrollador.

In [63]:
# Leer el archivo parquet, asegurándonos de que las columnas de fechas sean tratadas como tal
df4 = pd.read_parquet('etljuegoslimpio.parquet', columns=['id', 'developer', 'release_date', 'price'])

# Convertir 'release_date' a tipo datetime
df4['release_date'] = pd.to_datetime(df4['release_date'], errors='coerce')

# Asegurarse de que 'developer' esté en minúsculas
df4['developer'] = df4['developer'].apply(lambda x: x.lower())

# Crear la columna 'free' basada en el precio
df4['free'] = df4['price'].apply(lambda x: 1 if x == 0 else 0)

# Crear la columna 'Año' extrayendo el año de 'release_date'
df4['Año'] = df4['release_date'].dt.year

# Agrupar por 'developer' y 'Año', y calcular la suma de 'id' y el porcentaje de contenido gratuito
df4 = df4.groupby(['developer', 'Año']).agg(
    {'id': 'count', 'free': lambda x: (x.sum() / x.count()) * 100}
).reset_index().rename(columns={'free': 'Contenido Free', 'id': 'Cantidad de Items'})

# Redondear la columna 'Contenido Free' a dos decimales
df4['Contenido Free'] = round(df4['Contenido Free'], 2)

# Mostrar las primeras filas del DataFrame
print(df4.head())


            developer   Año  Cantidad de Items  Contenido Free
0         +7 software  2016                  1             0.0
1  +mpact games, llc.  2017                  1             0.0
2           .ez games  2017                  1             0.0
3             .m.y.w.  2016                  1             0.0
4      07th expansion  2015                  2             0.0


In [35]:
df4.to_parquet('desarrollador.parquet')

In [51]:
df4.tail()

,developer,Año,Cantidad de Items,Contenido Free
15033,萌石游戏,1970-01-01 00:00:00.000002017,1,0.0
15034,高考恋爱委员会,1970-01-01 00:00:00.000002015,1,100.0
15035,"高考恋爱委员会,days",1970-01-01 00:00:00.000002015,1,0.0
15036,"高考恋爱委员会,橘子班",1970-01-01 00:00:00.000002015,1,0.0
15037,"高考恋爱委员会,橘子班",1970-01-01 00:00:00.000002017,1,0.0


In [55]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15038 entries, 0 to 15037
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   developer          15038 non-null  object        
 1   Año                15038 non-null  datetime64[ns]
 2   Cantidad de Items  15038 non-null  int64         
 3   Contenido Free     15038 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 470.1+ KB


In [53]:
# Convertir la columna 'fechas' a formato datetime
df4['Año'] = pd.to_datetime(df4['Año'], errors='coerce')

print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [60]:
def developer(desarrollador: str):
    
    """
    Recupera información sobre un desarrollador específico.

    Args:
        desarrollador (str): Nombre del desarrollador.

    Returns:
        dict: Diccionario con información sobre el desarrollador.
    """
    
    if not isinstance(desarrollador, str):
        return {'Mensaje': 'El argumento "desarrollador" debe ser una cadena de texto (str).'}
    
    df4 = pd.read_parquet('desarrollador.parquet')

    desarrollador = desarrollador.lower()
    desarrolladores = list(df4['developer'].unique())
    df4 = df4[df4['developer'].isin([desarrollador])].reset_index(drop=True)

    if len(df4) == 0:
        del df4
        return {'Mensaje': f'Desarrollador no encontrado. Inserte un desarrollador válido. Desarrolladores: {", ".join(desarrolladores)}'}

    resultado= {
        f'Año {int(df4.loc[i,"Año"])}': {'Cantidad de Items': int(df4.loc[i, 'Cantidad de Items']), 'Contenido Free:': str(df4.loc[i, 'Contenido Free']) + ' %'}
        for i in range(len(df4))
    }

    del df4

In [62]:
def developer(desarrollador: str):
    """
    Recupera información sobre un desarrollador específico.
    
    Args:
        desarrollador (str): Nombre del desarrollador.
    
    Returns:
        dict: Diccionario con información sobre el desarrollador.
    """
    
    if not isinstance(desarrollador, str):
        return {'Mensaje': 'El argumento "desarrollador" debe ser una cadena de texto (str).'}
    
    # Cargar datos desde el archivo parquet
    df4 = pd.read_parquet('desarrollador.parquet')
    
    # Convertir el nombre del desarrollador a minúsculas
    desarrollador = desarrollador.lower()
    
    # Obtener lista de desarrolladores únicos
    desarrolladores = list(df4['developer'].unique())
    
    # Filtrar DataFrame por el desarrollador especificado
    df4 = df4[df4['developer'].isin([desarrollador])].reset_index(drop=True)
    
    if len(df4) == 0:
        del df4
        return {'Mensaje': f'Desarrollador no encontrado. Inserte un desarrollador válido. Desarrolladores: {", ".join(desarrolladores)}'}
    
    # Preparar el diccionario de resultados
    resultado = {
        f'Año {int(df4.loc[i, "Año"])}': {
            'Cantidad de Items': int(df4.loc[i, 'Cantidad de Items']),
            'Contenido Free': str(df4.loc[i, 'Contenido Free']) + ' %'
        }
        for i in range(len(df4))
    }
    
    del df4
    
    return resultado

# Ejemplo de uso
print(developer('valve'))


{'Año 1998': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 1999': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2000': {'Cantidad de Items': 2, 'Contenido Free': '0.0 %'}, 'Año 2001': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2003': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2004': {'Cantidad de Items': 5, 'Contenido Free': '0.0 %'}, 'Año 2005': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2006': {'Cantidad de Items': 2, 'Contenido Free': '0.0 %'}, 'Año 2007': {'Cantidad de Items': 3, 'Contenido Free': '33.33 %'}, 'Año 2008': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2009': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2010': {'Cantidad de Items': 2, 'Contenido Free': '50.0 %'}, 'Año 2011': {'Cantidad de Items': 1, 'Contenido Free': '0.0 %'}, 'Año 2012': {'Cantidad de Items': 2, 'Contenido Free': '0.0 %'}, 'Año 2014': {'Cantidad de Items': 7, 'Contenido Free': '0.0 %'}, 'Año 2016': {'Cantida

## Función User Data

In [68]:
reseñas=pd.read_parquet('reseñas.parquet')

In [85]:
reseñas.head()

,user_id,review,item_id,Recommend,Año,año_modaitem
0,76561197970982479,Simple yet with great replayability. In my opi...,1250,True,2011.0,2014.0
1,76561197970982479,It's unique and worth a playthrough.,22200,True,2011.0,2011.0
2,76561197970982479,Great atmosphere. The gunplay can be a bit chu...,43110,True,2011.0,2014.0
3,js41637,I know what you think when you see this title ...,251610,True,2014.0,2014.0
4,js41637,For a simple (it's actually not all that simpl...,227300,True,2013.0,2014.0


In [70]:
reseñas.drop(columns='0', inplace=True)

In [87]:
reseñas.rename(columns={'Recommend':'recommend'}, inplace=True)

In [88]:
reseñas.head()

,user_id,review,item_id,recommend,Año,año_modaitem
0,76561197970982479,Simple yet with great replayability. In my opi...,1250,True,2011.0,2014.0
1,76561197970982479,It's unique and worth a playthrough.,22200,True,2011.0,2011.0
2,76561197970982479,Great atmosphere. The gunplay can be a bit chu...,43110,True,2011.0,2014.0
3,js41637,I know what you think when you see this title ...,251610,True,2014.0,2014.0
4,js41637,For a simple (it's actually not all that simpl...,227300,True,2013.0,2014.0


In [89]:
reseñas.to_parquet('reseñas.parquet')

In [91]:
# Cargar los datos
df_user = pd.read_parquet('items_usuarios.parquet', columns=['user_id', 'item_id', 'items_count'])
df_steam = pd.read_parquet('etljuegoslimpio.parquet', columns=['id', 'price'])

# Convertir los tipos de datos de las columnas 'item_id' y 'id' a string para asegurar compatibilidad
df_user['item_id'] = df_user['item_id'].astype(str)
df_steam['id'] = df_steam['id'].astype(str)

# Unir los DataFrames
df5 = df_user.merge(df_steam, how='left', left_on='item_id', right_on='id')

# Eliminar columnas innecesarias y liberar memoria
del df_user
del df_steam
df5.drop(columns=['item_id', 'id'], inplace=True)

# Agrupar por 'user_id' y calcular el máximo de 'items_count' y la suma de 'price'
df5 = df5.groupby('user_id').agg({'items_count': 'max', 'price': 'sum'}).reset_index()
df5.rename(columns={'price': 'dinero_gastado'}, inplace=True)

# Cargar y agrupar las reseñas por 'user_id'
df_reviews = pd.read_parquet('reseñas.parquet', columns=['user_id', 'recommend'])
df_reviews = df_reviews.groupby('user_id').agg('sum').reset_index()

# Unir los DataFrames
df6 = df5.merge(df_reviews, how='left')

# Eliminar DataFrame innecesario y calcular el porcentaje de recomendaciones
del df_reviews
df6['porcentaje_recom'] = round(df6['recommend'] * 100 / df6['items_count'], 2)

# Limpiar el DataFrame final
df6.drop(columns='recommend', inplace=True)
df6.fillna(0, inplace=True)

# Mostrar las primeras filas del DataFrame final
print(df6.head())

# Guardar el DataFrame final (comentado para evitar errores no deseados)
df6.to_parquet('userdata.parquet')


             user_id  items_count  dinero_gastado  porcentaje_recom
0            --000--           58             0.0              1.72
1            --ace--           44             0.0              4.55
2          --ionex--           23             0.0              8.70
3       -2SV-vuLB-Kg           68             0.0              7.35
4  -404PageNotFound-          149             0.0              0.00


In [95]:
def userdata(user_id: str):

    if not isinstance(user_id, str):
        return {'Mensaje': 'El argumento user_id debe ser una cadena de texto.'}

    df7 = pd.read_parquet('userdata.parquet')
    df7= df7[df7['user_id'].isin([user_id])].reset_index(drop=True)
    
    if df7.empty:
        del df7
        return {'Mensaje': 'Usuario no encontrado. Por favor ingrese un usuario válido'}
    
    df7 = df7.reset_index(drop=True)

    resultado = {
        'Usuario': user_id,
        'Dinero gastado': str(round(df7.loc[0,'dinero_gastado'], 2)) + ' USD',
        '% de recomendación': str(df7.loc[0,'porcentaje_recom']) + ' %',
        'Cantidad de Items': int(df7.loc[0,'items_count'])
    }

    return resultado

In [98]:
userdata('--ionex--')

{'Usuario': '--ionex--',
 'Dinero gastado': '0.0 USD',
 '% de recomendación': '8.7 %',
 'Cantidad de Items': 23}

## Usuario por género

In [4]:
df_genres = pd.read_parquet('generos.parquet', columns=['genres', 'id', 'release_date'])
df_genres.head()

,genres,id,release_date
0,Strategy,761140,2018-01-04
1,Action,761140,2018-01-04
2,Indie,761140,2018-01-04
3,Casual,761140,2018-01-04
4,Simulation,761140,2018-01-04


In [5]:
df_user = pd.read_parquet('items_usuarios.parquet', columns=['user_id', 'playtime_forever', 'item_id'])
df_user.head()

,user_id,playtime_forever,item_id
0,76561197970982479,6,10
1,76561197970982479,0,20
2,76561197970982479,7,30
3,76561197970982479,0,40
4,76561197970982479,0,50


In [6]:
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155382 entries, 0 to 155381
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   genres        155268 non-null  object        
 1   id            155382 non-null  int64         
 2   release_date  155382 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.6+ MB


In [7]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   playtime_forever  int64 
 2   item_id           object
dtypes: int64(1), object(2)
memory usage: 155.5+ MB


In [8]:
df_genres['id'] = df_genres['id'].astype(int)

In [9]:
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155382 entries, 0 to 155381
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   genres        155268 non-null  object        
 1   id            155382 non-null  int64         
 2   release_date  155382 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.6+ MB


In [10]:
df_user['item_id'] = df_user['item_id'].astype(int)

In [11]:
df_user.head()

,user_id,playtime_forever,item_id
0,76561197970982479,6,10
1,76561197970982479,0,20
2,76561197970982479,7,30
3,76561197970982479,0,40
4,76561197970982479,0,50


In [23]:
df8 = df_genres.merge(df_user, how='left', left_on='id', right_on='item_id')

print(df8.head())


       genres      id release_date user_id  playtime_forever  item_id
0    Strategy  761140   2018-01-04     NaN               NaN      NaN
1      Action  761140   2018-01-04     NaN               NaN      NaN
2       Indie  761140   2018-01-04     NaN               NaN      NaN
3      Casual  761140   2018-01-04     NaN               NaN      NaN
4  Simulation  761140   2018-01-04     NaN               NaN      NaN


In [24]:
df8.isna().sum()

genres                167
id                      0
release_date            0
user_id             80121
playtime_forever    80121
item_id             80121
dtype: int64

In [25]:
df8.dropna(inplace=True)

In [26]:
df8.isna().sum()

genres              0
id                  0
release_date        0
user_id             0
playtime_forever    0
item_id             0
dtype: int64

In [27]:
df8.head()

,genres,id,release_date,user_id,playtime_forever,item_id
108,Racing,282010,1997-06-30,UTNerd24,5.0,282010.0
109,Racing,282010,1997-06-30,I_DID_911_JUST_SAYING,0.0,282010.0
110,Racing,282010,1997-06-30,76561197962104795,0.0,282010.0
111,Racing,282010,1997-06-30,r3ap3r78,0.0,282010.0
112,Racing,282010,1997-06-30,saint556,13.0,282010.0


In [28]:
df8['Año'] = df8['release_date'].dt.year

In [29]:
df8.drop(columns=['id', 'item_id', 'release_date'], inplace=True)
df8.head()

,genres,user_id,playtime_forever,Año
108,Racing,UTNerd24,5.0,1997
109,Racing,I_DID_911_JUST_SAYING,0.0,1997
110,Racing,76561197962104795,0.0,1997
111,Racing,r3ap3r78,0.0,1997
112,Racing,saint556,13.0,1997


In [30]:
df8 = df8.groupby(['genres', 'user_id', 'Año']).agg('sum').reset_index()
df8.head()

In [31]:
df8['playtime_forever'] = round(df8['playtime_forever'] / 60, 2)

In [36]:
df8['genres'] = df8['genres'].apply(lambda x: x.lower())
df8.tail()

,genres,user_id,Año,playtime_forever
33452387,e-sports,zzyfo,2000,0.08
33452388,e-sports,zzyfo,2004,3.38
33452389,e-sports,zzyfo,2010,0.38
33452390,e-sports,zzyfo,2012,155.13
33452391,e-sports,zzyfo,2015,0.77


In [37]:
df8.tail()

,genres,user_id,Año,playtime_forever
33452387,e-sports,zzyfo,2000,0.08
33452388,e-sports,zzyfo,2004,3.38
33452389,e-sports,zzyfo,2010,0.38
33452390,e-sports,zzyfo,2012,155.13
33452391,e-sports,zzyfo,2015,0.77


In [33]:
df8.to_parquet('userporgenero.parquet')

In [43]:
def UserForGenre(genero: str):
    
    if not isinstance(genero, str):
        return {'Mensaje': 'El género ingresado debe ser una cadena de texto (string)'}
    
    genero = genero.lower()

    try:
        df8 = pd.read_parquet('userporgenero.parquet')
        df8 = df8[df8['genres'].isin(['action'])].drop(columns='genres')
    except Exception:
        return {'Error': 'Género no encontrado. Ingrese un género válido'}

    usuario_max_horas = df8.groupby('user_id').agg({'playtime_forever': 'sum'}).sort_values('playtime_forever').tail(1).index[0]
    df8 = df8[df8['user_id'].isin([usuario_max_horas])].reset_index(drop=True)

    resultado = {
        f'Usuario con mas horas jugadas para el género {genero}:': usuario_max_horas,
        'Horas jugadas:': [{'Año:': int(df8.loc[i,'Año']), 'Horas:': float(df8.loc[i,'playtime_forever'])} for i in range(len(df8))]
    }
    del df8
    return resultado

In [44]:
UserForGenre('e-sports')

{'Usuario con mas horas jugadas para el género e-sports:': 'Sp3ctre',
 'Horas jugadas:': [{'Año:': 1993, 'Horas:': 0.0},
  {'Año:': 1995, 'Horas:': 3.62},
  {'Año:': 1996, 'Horas:': 0.0},
  {'Año:': 1997, 'Horas:': 4.03},
  {'Año:': 1998, 'Horas:': 0.0},
  {'Año:': 1999, 'Horas:': 0.73},
  {'Año:': 2000, 'Horas:': 1177.4},
  {'Año:': 2001, 'Horas:': 4.17},
  {'Año:': 2002, 'Horas:': 3.97},
  {'Año:': 2003, 'Horas:': 129.07},
  {'Año:': 2004, 'Horas:': 2123.52},
  {'Año:': 2005, 'Horas:': 355.65},
  {'Año:': 2006, 'Horas:': 1504.22},
  {'Año:': 2007, 'Horas:': 1879.9},
  {'Año:': 2008, 'Horas:': 142.22},
  {'Año:': 2009, 'Horas:': 1812.85},
  {'Año:': 2010, 'Horas:': 1379.43},
  {'Año:': 2011, 'Horas:': 2590.98},
  {'Año:': 2012, 'Horas:': 6431.58},
  {'Año:': 2013, 'Horas:': 2035.97},
  {'Año:': 2014, 'Horas:': 2214.4},
  {'Año:': 2015, 'Horas:': 6571.88},
  {'Año:': 2016, 'Horas:': 498.62},
  {'Año:': 2017, 'Horas:': 722.12}]}